In [1]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import sympy as sp
from mpl_toolkits.mplot3d import Axes3D

with open("output.pkl", "rb") as file:
    dVr = pickle.load(file)
    dVp = pickle.load(file)

dVr

(-3*M*V1**2*r**2 + M*(2*M - r)**2 - V3**2*r**3*(2*M - r)**2)/(r**3*(2*M - r))

In [2]:
dvr = sp.lambdify(
    (sp.Symbol("V1"), sp.Symbol("V3"), sp.Symbol("r"), sp.Symbol("M")), dVr, "numpy"
)
dvp = sp.lambdify(
    (sp.Symbol("V1"), sp.Symbol("V3"), sp.Symbol("r"), sp.Symbol("M")), dVp, "numpy"
)

In [3]:
from mayavi import mlab
from numpy import cos, pi, sin, tan
from scipy.integrate import solve_ivp

#mlab.init_notebook()

M = 1


def geodesic(t, state):
    r, theta, phi, V1, V2, V3 = state

    dr = V1
    dth = 0
    dph = V3
    dV1 = dvr(V1, V3, r, 1)
    dV2 = 0
    dV3 = dvp(V1, V3, r, 1)

    return np.array([dr, dth, dph, dV1, dV2, dV3])

In [4]:
# Constants

# y0 = [4, pi/2, 0, 0, 0, 0.125] : Circular motion
#[4, np.pi/2, 0, -0.1, 0 , 0.143498197]
bh_r = 2 * M
y0 = np.array([20, pi/2, 0, -0.1, 0 , 0.01])

t = 10000
delay = int(t / 100)
t_span = (0.0, t)
t_eval = np.linspace(0, t, t + 1)

bh_phi, bh_theta = np.mgrid[0:pi:101j, 0 : 2 * pi : 101j]
bh_x = bh_r * sin(bh_phi) * cos(bh_theta)
bh_y = bh_r * sin(bh_phi) * sin(bh_theta)
bh_z = bh_r * cos(bh_phi)

fig = mlab.gcf()
mlab.clf()


black_hole = mlab.mesh(bh_x, bh_y, bh_z, color=(0, 0, 0))

qt.qpa.window: <QNSWindow: 0x7fa4382de480; contentView=<QNSView: 0x7fa4382de080; QCocoaWindow(0x60000167a730, window=QWidgetWindow(0x6000005164c0, name="QMainWindowClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.


In [5]:
#%%
def lim_fun(t,y):
    return (y[0] - 2) > 2E-15

    
lim_fun.terminal = True 

result_solve_ivp = solve_ivp(
    geodesic, t_span, y0, rtol=1e-14, atol=1e-15, method="RK45", t_eval=t_eval,
                                          events=lim_fun
)

/Users/aliosmanyilmaz/opt/anaconda3/lib/python3.9/site-packages/scipy/integrate/_ivp/common.py:47: UserWarning: At least one element of `rtol` is too small. Setting `rtol = np.maximum(rtol, 2.220446049250313e-14)`.
  warn("At least one element of `rtol` is too small. "


In [6]:
g_r, g_phi = np.meshgrid(np.geomspace(2.01, 10, num=400), np.linspace(0, 2 * pi, 101))

g_x = g_r * cos(g_phi)
g_y = g_r * sin(g_phi)


ss = np.sqrt(1 / (1 - 2 / np.sqrt(g_x**2 + g_y**2)))
ss[np.isinf(ss) | np.isnan(ss)] = 0

g_z = (-ss + 1)
norm = g_z.min()
g_z = -g_z/norm*8

eq_surf = mlab.mesh(g_x, g_y, g_z, scalars=ss, opacity=0.4, colormap="blue-red")

In [7]:
#%%
r = result_solve_ivp.y[0]
theta = result_solve_ivp.y[1]
phi = result_solve_ivp.y[2]

x = r * cos(phi)
y = r * sin(phi)
z = (-np.sqrt(1 / (1 - 2 / np.sqrt(x**2 + y**2))) + 1)
z = -z/norm*8



z[z<g_z.min()] = g_z.min()

color = z.copy()

orbit = mlab.plot3d(x, y, z, tube_radius=0.05, opacity=1, color=(0.3, 0.4, 0.5))

#mlab.show()
#mlab.gcf()

In [8]:
colors = -color
colors = colors/colors.max()

planet = mlab.points3d(x[0], y[0], z[0], color=(1,.01,.01))

In [9]:
#%%



@mlab.animate(delay=1000)
def anim():
    fig = mlab.gcf()
    for ii in np.arange(0, len(x)):
        planet.mlab_source.trait_set(x=x[ii], y=y[ii], z=z[ii])
        if ii > 0:
            orbit.mlab_source.reset(x=x[:ii], y=y[:ii], z=z[:ii])
        yield
        fig.scene.reset_zoom()


anim()
mlab.view(0, 180)
mlab.show()

AttributeError: 'NoneType' object has no attribute 'reset_zoom'

In [ ]:
mmm = z.copy()

In [ ]:
colors[-1]